# Research Workspace Demo

This notebook demonstrates the complete research workflow:
1. Load data
2. Compute features
3. Backtest strategy
4. Visualize results

This shows how to use the research utilities for rapid strategy development.

In [ ]:
# Setup - Add project to path
import sys
sys.path.append('/Users/jacobliu/repos/projects/trading-bot')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import research utilities
from research.utils import (
    QuickBacktest,
    load_sample_data,
    plot_equity_curve,
    plot_signals,
    plot_correlation_matrix,
    calculate_metrics,
)
from research.utils.data_loaders import add_features

%matplotlib inline
%load_ext autoreload
%autoreload 2

print("Research workspace loaded successfully!")

## Step 1: Load Market Data

Use the convenient `load_sample_data()` function to generate test data.

In [ ]:
# Load sample data for SPY
data = load_sample_data('SPY', days=500)

# Prepare data for analysis
if 'symbol' in data.columns:
    data = data[data['symbol'] == 'SPY'].set_index('timestamp').sort_index()

print(f"Loaded {len(data)} bars")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"\nFirst few rows:")
data.head()

## Step 2: Compute Features

Use `add_features()` to quickly add technical indicators.

In [ ]:
# Add technical features
data = add_features(data, features=['returns', 'sma', 'ema', 'rsi', 'bbands'])

# Add custom features
data['sma_cross'] = (data['sma_10'] > data['sma_50']).astype(int)
data['momentum'] = data['close'].pct_change(20)

# Remove NaN rows
data = data.dropna()

print(f"Features computed: {list(data.columns)}")
print(f"\nData shape: {data.shape}")
data.tail()

## Step 3: Visualize Feature Relationships

In [ ]:
# Correlation matrix of key features
features_to_analyze = ['returns', 'rsi_14', 'momentum', 'bb_width']
plot_correlation_matrix(data[features_to_analyze], title='Feature Correlations')

## Step 4: Implement a Simple Strategy

Let's create a simple RSI mean-reversion strategy.

In [ ]:
# Generate trading signals based on RSI
data['signal'] = 0.0

# Buy when RSI < 30 (oversold)
data.loc[data['rsi_14'] < 30, 'signal'] = 1.0

# Sell when RSI > 70 (overbought)
data.loc[data['rsi_14'] > 70, 'signal'] = 0.0

# Forward fill signals (hold position)
data['signal'] = data['signal'].replace(0.0, np.nan).ffill().fillna(0.0)

print(f"Generated {data['signal'].sum():.0f} buy signals")
print(f"Position changes: {(data['signal'].diff() != 0).sum()}")

## Step 5: Visualize Signals

In [ ]:
# Plot signals on price chart
buy_signals = (data['signal'].diff() > 0)
sell_signals = (data['signal'].diff() < 0)

plot_signals(
    data.iloc[-200:],  # Last 200 days
    buy_signals=buy_signals.iloc[-200:],
    sell_signals=sell_signals.iloc[-200:],
    show_features=['rsi_14'],
    title='RSI Mean Reversion Strategy'
)

## Step 6: Backtest the Strategy

Use `QuickBacktest` for fast strategy testing.

In [ ]:
# Initialize backtester
bt = QuickBacktest(
    initial_capital=100000,
    commission=0.001,  # 10 bps
    slippage=0.0005,   # 5 bps
)

# Run backtest
result = bt.run_signals(
    data,
    signals=data['signal'],
    position_size=1.0,  # Use 100% of capital
    verbose=True
)

print("\n" + "="*60)
print("BACKTEST RESULTS")
print("="*60)
print(result)

## Step 7: Comprehensive Results Visualization

In [ ]:
# Plot comprehensive results
result.plot()

## Step 8: Detailed Performance Analysis

In [ ]:
# Calculate additional metrics
metrics = calculate_metrics(result.returns)

print("\nDetailed Performance Metrics:")
print("="*60)
for key, value in metrics.items():
    if isinstance(value, float):
        if 'return' in key or 'win' in key or 'drawdown' in key:
            print(f"{key:20s}: {value:>10.2%}")
        else:
            print(f"{key:20s}: {value:>10.4f}")
    else:
        print(f"{key:20s}: {value:>10}")

In [ ]:
# Plot equity curve with drawdown
plot_equity_curve(
    result.equity_curve,
    title='Strategy Performance',
    show_drawdown=True
)

## Step 9: Compare with Buy & Hold

In [ ]:
# Create buy & hold benchmark
buy_hold_signal = pd.Series(1.0, index=data.index)
result_bh = bt.run_signals(data, buy_hold_signal, position_size=1.0)

# Compare strategies
comparison = pd.DataFrame({
    'RSI Strategy': result.equity_curve,
    'Buy & Hold': result_bh.equity_curve,
})

plot_equity_curve(comparison, title='Strategy vs Buy & Hold')

# Print comparison
print("\nStrategy Comparison:")
print("="*60)
comparison_metrics = pd.DataFrame({
    'RSI Strategy': result.metrics,
    'Buy & Hold': result_bh.metrics,
}).T

print(comparison_metrics[['total_return', 'sharpe_ratio', 'max_drawdown', 'num_trades']])

## Step 10: Rolling Performance Analysis

In [ ]:
from research.utils.analysis import rolling_metrics

# Calculate rolling metrics
rolling = rolling_metrics(result.returns, window=63, metrics=['sharpe', 'vol', 'win_rate'])

# Plot rolling Sharpe
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(rolling.index, rolling['sharpe'], linewidth=2, label='Rolling Sharpe (63d)')
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.axhline(y=rolling['sharpe'].mean(), color='red', linestyle='--',
          label=f'Average: {rolling["sharpe"].mean():.2f}')
ax.set_title('Rolling Sharpe Ratio', fontweight='bold', fontsize=14)
ax.set_ylabel('Sharpe Ratio')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This demo showed how to:

1. **Load data** with `load_sample_data()` or `load_bars()`
2. **Add features** with `add_features()`
3. **Generate signals** using feature-based logic
4. **Backtest quickly** with `QuickBacktest`
5. **Visualize results** with rich plotting functions
6. **Analyze performance** with comprehensive metrics

## Next Steps

1. Use `signal_discovery.ipynb` template to explore new ideas
2. Use `strategy_validation.ipynb` to rigorously test strategies
3. Use `feature_engineering.ipynb` to develop new features
4. When ready, move to production engine for real backtesting

Remember: QuickBacktest is for **rapid prototyping only**. 
Use the full simulation engine for production-quality backtesting!